# Module 2: A Bad Plan is better Than no Plan
## Making AI Think and Remember

### 1. Introduction to AI Planning
One way to build apps with semantic kernel would be to use the concepts of module 1 and orchestrate those with code... But what if instead of you having to figure out how to chain functions together, the planner can look at all available tools and figure out the best way to solve a problem.

### 2. Types of Planners

#### 2.1 Sequential Planner
The Sequential Planner is like a cooking recipe - it figures out the steps one after another:

In [ ]:
from semantic_kernel.planners import SequentialPlanner
from semantic_kernel.core_plugins import TextPlugin
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt

# Set up kernel with OpenAI
kernel = Kernel()
service_id = "default"
kernel.add_service(
    OpenAIChatCompletion(
        service_id=service_id,
    ),
)

# Add the built-in text plugin
text_plugin = kernel.add_plugin(TextPlugin(), "text")

# Create a custom semantic function for creative writing
story_function = KernelFunctionFromPrompt(
    function_name="WriteStory",
    plugin_name="WriterPlugin",
    prompt="""
    Write a short story about: {{$input}}
    Make it creative and engaging.
    """,
    description="Writes a creative short story based on input."
)
kernel.add_function(plugin_name="WriterPlugin", function=story_function)

# Create French translation function
translate_function = KernelFunctionFromPrompt(
    function_name="TranslateToFrench",
    plugin_name="WriterPlugin",
    prompt="""
    Translate the following text to French:
    {{$input}}
    """,
    description="Translates text to French."
)
kernel.add_function(plugin_name="WriterPlugin", function=translate_function)

# Create our planner
planner = SequentialPlanner(kernel, service_id)

# Create a plan for a complex task
ask = """
Write a story about a programmer solving a mysterious bug,
translate it to French, and then convert it to uppercase.
"""

plan = await planner.create_plan(goal=ask)

print("The plan's steps are:")
for step in plan._steps:
    print(f"- {step.description} using {step.plugin_name}.{step.name}")

result = await plan.invoke(kernel)
print("\nFinal Result:")
print(result)

#### 2.2 Function Calling Stepwise Planner
The Stepwise Planner can think and observe as it executes. It's particularly good at tasks that need reasoning:

In [ ]:
from typing import Annotated
from semantic_kernel.planners.function_calling_stepwise_planner import (
    FunctionCallingStepwisePlanner,
    FunctionCallingStepwisePlannerOptions,
)
from semantic_kernel.core_plugins import MathPlugin, TimePlugin

# Set up kernel with plugins we'll use
kernel = Kernel()
kernel.add_service(
    OpenAIChatCompletion(
        service_id="default",
    ),
)

# Add some useful plugins
kernel.add_plugin(plugin_name="math", plugin=MathPlugin())
kernel.add_plugin(plugin_name="time", plugin=TimePlugin())

# Create email plugin (as shown in SK examples)
class EmailPlugin:
    @kernel_function(
        name="SendEmail",
        description="Given an e-mail and message body, send an e-email"
    )
    def send_email(
        self,
        subject: Annotated[str, "the subject of the email"],
        body: Annotated[str, "the body of the email"],
    ) -> str:
        return f"Email sent with subject: {subject} and body: {body}"

    @kernel_function(
        name="GetEmailAddress",
        description="Given a name, find the email address"
    )
    def get_email_address(
        self,
        input: Annotated[str, "the name of the person"],
    ):
        email_directory = {
            "Jane": "jane@example.com",
            "Paul": "paul@example.com",
            "Mary": "mary@example.com"
        }
        return email_directory.get(input, "unknown@example.com")

kernel.add_plugin(plugin_name="email", plugin=EmailPlugin())

# Create the planner with options
planner = FunctionCallingStepwisePlanner(
    service_id="default",
    options=FunctionCallingStepwisePlannerOptions(
        max_iterations=10,
        max_tokens=4000,
    )
)

# Example tasks that combine multiple functions
tasks = [
    "What time is it in UTC? Add 3 hours and email the result to Jane.",
    "Calculate 437 * 239, divide by 2, and send the result to both Mary and Paul.",
    "What hour will it be in 5 hours? Email this information to Jane with the subject 'Future Time'."
]

for task in tasks:
    print(f"\nTask: {task}")
    result = await planner.invoke(kernel, task)
    print(f"Result: {result.final_answer}")

### 3. Combining Planners and Memory

Let's revisit memory!
Finish implementing the next cell to import any kind of data you want into the memory (for example: docs or code from github, parse wikipedia articles etc)





In [ ]:
from semantic_kernel.memory import VolatileMemoryStore
from semantic_kernel.connectors.ai.open_ai import OpenAITextEmbedding
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory

# Set up memory system
memory_store = VolatileMemoryStore()
embeddings = OpenAITextEmbedding(
    service_id="default"
)
memory = SemanticTextMemory(memory_store, embeddings)

# TODO: Add some data to the memory

# TODO: Query the memory

**Semantic Skills**: AI-powered functions defined by prompts

In [ ]:
import json
import requests
from typing import Annotated
from bs4 import BeautifulSoup
from semantic_kernel.functions.kernel_function_decorator import kernel_function

class WebResearchPlugin:
    """Plugin for web research capabilities"""
    
    @kernel_function(
        name="SearchWeb",
        description="Searches the web using DuckDuckGo and returns relevant URLs"
    )
    def search_web(
        self,
        query: Annotated[str, "the search query"]
    ) -> str:
        # Using DuckDuckGo API (no key required)
        url = f"https://api.duckduckgo.com/?q={query}&format=json"
        response = requests.get(url)
        data = json.loads(response.text)
        
        # Extract top results
        results = []
        for result in data.get('Results', [])[:3]:
            results.append(result['FirstURL'])
            
        return json.dumps(results)

    @kernel_function(
        name="ExtractContent",
        description="Extracts main content from a webpage"
    )
    def extract_content(
        self,
        url: Annotated[str, "the webpage URL"]
    ) -> str:
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Remove scripts, styles, and navigation
            for element in soup(['script', 'style', 'nav', 'header', 'footer']):
                element.decompose()
            
            # Extract text
            text = soup.get_text(separator=' ', strip=True)
            # Limit text length
            return text[:2000] + "..." if len(text) > 2000 else text
        except Exception as e:
            return f"Error extracting content: {str(e)}"

class ResearchPlugin:
    """Plugin for analyzing and summarizing research"""
    
    def __init__(self, memory: SemanticTextMemory):
        self.memory = memory

    @kernel_function(
        name="SaveToMemory",
        description="Saves research information to semantic memory"
    )
    async def save_to_memory(
        self,
        content: Annotated[str, "the content to save"],
        topic: Annotated[str, "the research topic"]
    ) -> str:
        try:
            # Save to memory
            await self.memory.save_information(
                collection="research_data",
                text=content,
                description=f"Research on {topic}",
                additional_metadata=json.dumps({"topic": topic})
            )
            return "Content saved to memory"
        except Exception as e:
            return f"Error saving to memory: {str(e)}"

# Set up the kernel with our plugins
kernel = Kernel()
kernel.add_service(
    OpenAIChatCompletion(
        service_id="default"
    )
)

# Add our research plugins
kernel.add_plugin(plugin_name="web", plugin=WebResearchPlugin())
kernel.add_plugin(
    plugin_name="research",
    plugin=ResearchPlugin(memory)
)

# Create semantic functions for analysis
analyze_function = KernelFunctionFromPrompt(
    function_name="AnalyzeContent",
    plugin_name="ResearchPlugin",
    prompt="""
    Content: {{$input}}
    
    Analyze this content and extract key points. Focus on:
    1. Main concepts and ideas
    2. Key findings or statements
    3. Important relationships
    4. Credibility of information
    
    Format your response as bullet points.
    """,
    description="Analyzes and extracts key points from content."
)

summarize_function = KernelFunctionFromPrompt(
    function_name="CreateSummary",
    plugin_name="ResearchPlugin",
    prompt="""
    Research Topic: {{$topic}}
    Collected Information: {{recall 'research_data' topic}}
    
    Create a comprehensive summary that:
    1. Synthesizes the main findings
    2. Highlights key agreements and contradictions
    3. Identifies gaps in the information
    4. Suggests areas for further research
    
    Keep the summary clear and well-structured.
    """,
    description="Creates a summary from collected research."
)

kernel.add_function(plugin_name="ResearchPlugin", function=analyze_function)
kernel.add_function(plugin_name="ResearchPlugin", function=summarize_function)

# Create the stepwise planner
planner = FunctionCallingStepwisePlanner(
    service_id="default",
    options=FunctionCallingStepwisePlannerOptions(
        max_iterations=15,
        max_tokens=4000,
    )
)

# Example research task
research_task = """
Research the latest developments in quantum computing.
Focus on recent breakthroughs in error correction.
Analyze the information and create a summary.
Email the findings to jane@example.com with the subject 'Quantum Computing Research'.
"""

async def conduct_research(task: str):
    try:
        result = await planner.invoke(kernel, task)
        print("Research Results:")
        print(result.final_answer)
        
        # Print the thought process (optional)
        print("\nThought Process:")
        for thought in result.chat_history:
            print(f"- {thought}")
            
    except Exception as e:
        print(f"Error during research: {str(e)}")

# Run the research
await conduct_research(research_task)

## 5. Best Practices


Always provide clear, specific goals
Include error handling
Monitor plan execution steps
Validate results at each stage